In [ ]:
!pip install pyngrok

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 745 kB 4.3 MB/s 
  Created wheel for pyngrok: filename=pyngrok-5.1.0-py3-none-any.whl size=19007 sha256=3785d9e968255fd4a3da8156dec0f5cb159aab271a7f97865d7509afc6744642
  Stored in directory: /root/.cache/pip/wheels/bf/e6/af/ccf6598ecefecd44104069371795cb9b3afbcd16987f6ccfb3
Successfully built pyngrok


In [ ]:
!ngrok authtoken 2HFZwmSnRUIPXem5Q3tVRKEhL7j_3wfvfNz1oFsuH6BQwEUZf

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [ ]:
!pip install flask-wtf

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 136 kB 4.1 MB/s 


In [ ]:
from pyngrok import ngrok# Open a HTTP tunnel on the default port 80
public_url = ngrok.connect()

In [ ]:
public_url

<NgrokTunnel: "http://ab46-104-155-214-185.ngrok.io" -> "http://localhost:80">

In [ ]:
ngrok.kill()

The code is using the Flask framework to create a web application. The user can upload a video file using a form, which is then saved to the server. The video file is then processed to extract 22 frames from the video using OpenCV. The frames are then cropped to only show the mouth using the Dlib library. The cropped frames are saved to a directory on the server where they are again loaded and passed through the model to predict the class.

In [ ]:
from flask import Flask, render_template
from flask_wtf import FlaskForm
from wtforms import FileField, SubmitField
from werkzeug.utils import secure_filename
import os
from wtforms.validators import InputRequired
# from moviepy.editor import VideoFileClip
import numpy as np
import os
import imutils
import dlib
import math
import cv2 
import imageio
from imutils import face_utils
from tensorflow.keras.models import Model
from tensorflow.keras.models import load_model
from skimage.transform import resize
import time


# extracing frames from video using openCV
# limiting the frames to 22
def extract_frames(video_file,dir):
    vidcap = cv2.VideoCapture(video_file)

    # The function returns a Boolean value indicating if the frame/s was successfully read and an image matrix. 
    success, image = vidcap.read()
    idx = 0

    # Retrieves the total number of frames. The argument "cv2.CAP_PROP_FRAME_COUNT" specifies the property to be retrieved.
    possible_frames = vidcap.get(cv2.CAP_PROP_FRAME_COUNT)
    gap = math.floor(possible_frames/22)
    os.makedirs(dir,exist_ok=True)
    print(gap)
    i = 0
    success, image = vidcap.read()
    while success:
      # limiting to 22 frames
        if(idx % gap == 0):
            cv2.imwrite(f"{dir}/frame{i:03d}.png", image)
            i += 1
        success, image = vidcap.read()
        if(i == 22 ):
            break
        idx += 1

app = Flask(__name__,template_folder='templates')
app.config['SECRET_KEY'] = 'supersecretkey'
app.config['UPLOAD_FOLDER'] = 'static/files'

class UploadFileForm(FlaskForm):
    file = FileField("File", validators=[InputRequired()])
    submit = SubmitField("Upload File")

@app.route('/', methods=['GET',"POST"])
@app.route('/home', methods=['GET',"POST"])
def home():
    form = UploadFileForm()
    if form.validate_on_submit():
        file = form.file.data # First grab the file
        file.save('/content/static/files/previous.mp4') # Then save the file
        print('video saved')
        
        # extracting frames
  
        f_directory = '/content/static/frames'
        video_path = '/content/static/files/previous.mp4'
        extract_frames(video_path, f_directory)
        # to avoid getting names of other files such as .ipynb_checkpoint
        file_list = [_ for _ in os.listdir('/content/static/frames') if _.endswith('.png')]
        # os.chdir('../../')

        def crop_and_save_image(img, img_path, write_img_path, img_name):
            detector = dlib.get_frontal_face_detector()
            predictor = dlib.shape_predictor('/content/drive/MyDrive/datasets/shape_predictor_68_face_landmarks/shape_predictor_68_face_landmarks.dat')
            # load the input image, resize it, and convert it to grayscale

            image = cv2.imread(img_path)
            image = imutils.resize(image, width=500)
            gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

            # detect faces in the grayscale image
            rects = detector(gray, 1)
            if len(rects) > 1:
                print("Error")
                return
            if len(rects) < 1:
                print( "ERROR: no faces detected")
                return
            for (i, rect) in enumerate(rects):
                shape = predictor(gray, rect)
                shape = face_utils.shape_to_np(shape)
                name, i, j = 'mouth', 48, 68
                # clone = gray.copy()
                (x, y, w, h) = cv2.boundingRect(np.array([shape[i:j]]))        
                roi = gray[y:y+h, x:x+w]
                roi = imutils.resize(roi, width = 250, inter=cv2.INTER_CUBIC)        
                print( write_img_path)
                cv2.imwrite(write_img_path, roi)

        # os.mkdir('/content/static/cropped')
        os.makedirs('/content/static/cropped/', exist_ok=True)
        directory = '/content/static/frames/'
        dir_temp = '/content/static/cropped/'
        print('cropping frames')
        for img_name in file_list:
            image = imageio.imread(directory + '' + img_name)
            crop_and_save_image(image, directory + '' + img_name,dir_temp + '' + img_name, img_name)

        print('loading model')
        savedModel=load_model('/content/drive/MyDrive/models/cnn_adagrad_e45_bc16/model.h5')

        # preprocessing the cropped image
        max_seq_length = 22
        MAX_WIDTH = 100
        MAX_HEIGHT = 100


        sequence = []
        for img_name in file_list:        
            image = imageio.imread(dir_temp + '/' + img_name)
            image = resize(image, (MAX_WIDTH, MAX_HEIGHT))
            image = 255 * image
            # Convert to integer data type pixels.
            image = image.astype(np.uint8)
            sequence.append(image)                        
        pad_array = [np.zeros((MAX_WIDTH, MAX_HEIGHT))]                            
        sequence.extend(pad_array * (max_seq_length - len(sequence)))
        sequence = np.array(sequence)

        def normalize_it(X):
            v_min = X.min(axis=(2, 3), keepdims=True)
            v_max = X.max(axis=(2, 3), keepdims=True)
            X = (X - v_min)/(v_max - v_min)
            X = np.nan_to_num(X)
            return X

        
        # making prediction
        X_test = []
        X_test.append(sequence)
        X_test = np.array(X_test)
        X_test = normalize_it(X_test)
        X_test = np.expand_dims(X_test, axis=4)
        ypred = savedModel.predict(X_test)
        words = ['Begin', 'Choose', 'Connection', 'Navigation', 'Next', 'Previous', 'Start', 'Stop', 'Hello', 'Web']  
        predicted_words = [words[i] for i in np.argmax(ypred, axis=1)]
        print(predicted_words) 
        os.system("rm -rf /content/static/cropped")
        os.system("rm -rf /content/static/frames")
        return predicted_words[0]

    return render_template('index.html', form=form)

In [ ]:
app.run(debug=True,use_reloader=False,port=80)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


INFO:werkzeug: * Running on http://127.0.0.1:80/ (Press CTRL+C to quit)
INFO:werkzeug:127.0.0.1 - - [10/Nov/2022 09:43:08] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [10/Nov/2022 09:43:09] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [10/Nov/2022 09:56:26] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [10/Nov/2022 09:56:27] "GET /favicon.ico HTTP/1.1" 404 -
